In [3]:
# Importing libraries

from kaggle.api.kaggle_api_extended import KaggleApi
import numpy as np
import seaborn as sns 
import matplotlib as mpl
import pandas as pd

In [1]:
# Downloading the Dataset

api = KaggleApi()
api.authenticate()

api.dataset_download_files(
    "ngshiheng/michelin-guide-restaurants-2021",
    path=".",   
    unzip=True 
)

Dataset URL: https://www.kaggle.com/datasets/ngshiheng/michelin-guide-restaurants-2021


In [6]:
data = pd.read_csv('michelin_my_maps.csv')
data.head(3)

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,GreenStar,FacilitiesAndServices,Description
0,ES:SENZ,"Mietenkamer Straße 65, Grassau, 83224, Germany","Grassau, Germany",€€€€,"Creative, Modern Cuisine",12.465618,47.785630,4.986414e+11,https://guide.michelin.com/en/bayern/grassau/r...,https://www.das-achental.com/,3 Stars,0,"Air conditioning,Car park,Interesting wine list","Here in the restaurant of Das Achental hotel, ..."
1,Tohru in der Schreiberei,"Burgstraße 5, Munich, 80331, Germany","Munich, Germany",€€€€,"Modern Cuisine, Japanese Contemporary",11.577475,48.137597,4.989215e+11,https://guide.michelin.com/en/bayern/mnchen/re...,https://schreiberei-muc.de/,3 Stars,0,"Interesting wine list,Notable sake list",It is absolutely worth climbing the 23 steps o...
2,Schwarzwaldstube,"Tonbachstraße 237, Baiersbronn, 72270, Germany","Baiersbronn, Germany",€€€€,"Classic French, Creative",8.358280,48.536911,4.974425e+11,https://guide.michelin.com/en/baden-wurttember...,https://www.traube-tonbach.de/restaurants-bar/...,3 Stars,0,"Air conditioning,Car park,Great view,Interesti...","Schwarzwaldstube, the flagship restaurant of t..."


### Looks like there are some columns that will have no value for the analysis 
### Such as Url, WebsiteUrl,Address and PhoneNumber. Thus ,i will drop them

In [13]:
data = data.drop(['Url','WebsiteUrl','Address','PhoneNumber'],axis = 1)

In [16]:
data['Award'].value_counts()

Award
Selected Restaurants    11243
Bib Gourmand             3414
1 Star                   3097
2 Stars                   511
3 Stars                   157
Name: count, dtype: int64

### Hence, this guide lists 11,243 restaurants with the following classifications:

- **Bib Gourmand (3,414):** Restaurants recognized for good quality and good value, but **without Michelin stars**.  
- **1 Star (3,097):** “A very good restaurant in its category.”  
- **2 Stars (511):** “Excellent cooking, worth a detour.”  
- **3 Stars (157):** “Exceptional cuisine, worth a special journey.”

### 

In [14]:
data['Location'].value_counts()

Location
Tokyo, Japan                       551
Paris, France                      466
London, United Kingdom             349
Singapore                          299
New York, USA                      265
                                  ... 
Saint-Laurent-sur-Saône, France      1
Cliousclat, France                   1
Parçay-Meslay, France                1
Villecresnes, France                 1
San Roque, Spain                     1
Name: count, Length: 5847, dtype: int64